In [21]:
import pandas as pd

In [22]:
titanic_train = pd.read_csv("train.csv")
titanic_test = pd.read_csv("test.csv")
gender_submission = pd.read_csv("gender_submission.csv")

## Prepare the Data for Machine Learning Algorithms

### Data Cleaning

**Numeric values**

In [23]:
titanic_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [24]:
# check for na values
titanic_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [25]:
from sklearn.impute import SimpleImputer

In [26]:
# dropping na's in this column because this column is categorical
titanic_train = titanic_train.dropna(subset=['Embarked'])

In [27]:
imputer = SimpleImputer(strategy="median")

In [28]:
titanic_age_fare = titanic_train[["Age", "Fare"]]

In [29]:
imputer.fit(titanic_age_fare)

SimpleImputer(strategy='median')

In [30]:
# computed medians were stored in statistics_ attribute
imputer.statistics_ == titanic_age_fare.median()

Age     True
Fare    True
dtype: bool

In [31]:
titanic_imp_age_fare = imputer.transform(titanic_age_fare)

In [32]:
titanic_imp_age_fare

array([[22.    ,  7.25  ],
       [38.    , 71.2833],
       [26.    ,  7.925 ],
       ...,
       [28.    , 23.45  ],
       [26.    , 30.    ],
       [32.    ,  7.75  ]])

*If you want to put it back into a pandas DataFrame:*

In [33]:
#titatic_train_df = pd.DataFrame(X, columns=titanic_train_num.columns)

**Text and Categorical attributes**

In [34]:
titanic_string = titanic_train.select_dtypes(exclude='number')
titanic_string.head()

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,NaN,S


In [35]:
# check for value counts if there is outlier in categorical columns
print(titanic_string['Sex'].value_counts())
print(titanic_string['Embarked'].value_counts())

male      577
female    312
Name: Sex, dtype: int64
S    644
C    168
Q     77
Name: Embarked, dtype: int64


In [36]:
# check for null values 
print(titanic_string['Sex'].isnull().sum())
print(titanic_string['Embarked'].isnull().sum())

0
0


In [37]:
# selecting categorical columns
titanic_string_cat = titanic_string[['Sex', 'Embarked']]

In [38]:
titanic_string_cat.to_numpy()

array([['male', 'S'],
       ['female', 'C'],
       ['female', 'S'],
       ...,
       ['female', 'S'],
       ['male', 'C'],
       ['male', 'Q']], dtype=object)

*PS: for onehotencoder, hyperparamater handle_unknown='ignore' could be used instead of dropping na's from the Embarked  column at the bigging of the notebook. However, it can not be used simultaneously with drop paramater since we would be confused by the reason of zeros*

In [39]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(drop='if_binary')
enc.fit(titanic_string_cat)
enc.transform(titanic_string_cat).toarray()

array([[1., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [1., 1., 0., 0.],
       [1., 0., 1., 0.]])

 *Alternative to OneHotEncoder, especially when variables are in order like "good, normal, bad"*

In [40]:
# from sklearn.preprocessing import OrdinalEncoder

# ordinal_encoder = OrdinalEncoder()
# titanic_cats_encoded = ordinal_encoder.fit_transform(titanic_string_cat)
# titanic_cats_encoded[:5]
# titanic_cat_encoded.categories_

In [41]:
titanic_prepared = full_pipeline.fit_transform(titanc_train)

NameError: name 'full_pipeline' is not defined

In [48]:
%run pipeline.ipynb

male      577
female    312
Name: Sex, dtype: int64
S    644
C    168
Q     77
Name: Embarked, dtype: int64
0
0


NameError: name 'full_pipeline' is not defined

NameError: name 'full_pipeline' is not defined

NameError: name 'full_pipeline' is not defined